<a href="https://colab.research.google.com/github/zhukuixi/Fastai/blob/main/fastai_lesson4_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Kaggle Competition Data

In [3]:
import json
token = {"username":"kuixizhu","key":"dc7d6e998e4b5eb9417913d844e0df0b"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)
  


In [4]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Assign path
!kaggle config set -n path -v /content
# Download data
!kaggle competitions download -c us-patent-phrase-to-phrase-matching

- path is now set to: /content
  0% 0.00/682k [00:00<?, ?B/s]
100% 682k/682k [00:00<00:00, 108MB/s]


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
#!mv "/content/competitions/" "/content/gdrive/MyDrive"
!unzip /content/gdrive/MyDrive/competitions/us-patent-phrase-to-phrase-matching/us-patent-phrase-to-phrase-matching.zip -d /content/gdrive/MyDrive/competitions/us-patent-phrase-to-phrase-matching/

Mounted at /content/gdrive
Archive:  /content/gdrive/MyDrive/competitions/us-patent-phrase-to-phrase-matching/us-patent-phrase-to-phrase-matching.zip
replace /content/gdrive/MyDrive/competitions/us-patent-phrase-to-phrase-matching/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: None


## EDA

In [6]:
import pandas as pd
from pathlib import Path
path = Path('/content/gdrive/MyDrive/competitions/us-patent-phrase-to-phrase-matching')
!ls {path}

sample_submission.csv  train.csv
test.csv	       us-patent-phrase-to-phrase-matching.zip


In [7]:
df = pd.read_csv(path/"train.csv")
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [8]:
df.describe(include='object')


,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [9]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor
df.input.head()


0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

## Tokenization


In [10]:
!pip install -q datasets
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(df)

     |████████████████████████████████| 365 kB 7.9 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 212 kB 48.3 MB/s 
     |████████████████████████████████| 115 kB 50.7 MB/s 
     |████████████████████████████████| 127 kB 52.5 MB/s 


In [11]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [ ]:
model_nm = 'microsoft/deberta-v3-small'
!pip install transformers
!pip install sentencepiece

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
# Tokenize in a way fits the assigned model
tokz = AutoTokenizer.from_pretrained(model_nm)

In [16]:
def tok_func(x): return tokz(x["input"])

In [17]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [18]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [19]:
tokz.vocab['▁of']

265

In [20]:
tok_ds = tok_ds.rename_columns({'score':'labels'})


## Test and validation sets


In [21]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [22]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [23]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}



## Training


In [32]:
from transformers import TrainingArguments,Trainer
import numpy as np

In [28]:
bs = 128
epochs = 4
lr = 8e-5

设置模型参数

In [29]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

设置模型和Trainer

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [33]:
trainer.train();


The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, id, context, input, target. If anchor, id, context, input, target are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 856


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.023369,0.822092
2,No log,0.023218,0.831547
3,0.018100,0.022914,0.832146
4,0.018100,0.022815,0.831600


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, id, context, input, target. If anchor, id, context, input, target are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, id, context, input, target. If anchor, id, context, input, target are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, id, context, input, target. If anchor, id, context, input, target are not expected by `DebertaV2ForSequenceClassifica

In [38]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, id, context, input, target. If anchor, id, context, input, target are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 256


array([[ 0.48754883],
       [ 0.68554688],
       [ 0.58251953],
       [ 0.27807617],
       [-0.04443359],
       [ 0.46337891],
       [ 0.52246094],
       [-0.06744385],
       [ 0.22058105],
       [ 1.13964844],
       [ 0.27807617],
       [ 0.25024414],
       [ 0.74804688],
       [ 0.98046875],
       [ 0.75439453],
       [ 0.50683594],
       [ 0.30444336],
       [-0.02964783],
       [ 0.68115234],
       [ 0.34326172],
       [ 0.51269531],
       [ 0.21142578],
       [ 0.0916748 ],
       [ 0.19750977],
       [ 0.57373047],
       [-0.05239868],
       [-0.0512085 ],
       [-0.0526123 ],
       [-0.05209351],
       [ 0.60351562],
       [ 0.28857422],
       [-0.0036087 ],
       [ 0.70849609],
       [ 0.48413086],
       [ 0.41699219],
       [ 0.28710938]])

In [39]:
preds = np.clip(preds, 0, 1)
preds


array([[0.48754883],
       [0.68554688],
       [0.58251953],
       [0.27807617],
       [0.        ],
       [0.46337891],
       [0.52246094],
       [0.        ],
       [0.22058105],
       [1.        ],
       [0.27807617],
       [0.25024414],
       [0.74804688],
       [0.98046875],
       [0.75439453],
       [0.50683594],
       [0.30444336],
       [0.        ],
       [0.68115234],
       [0.34326172],
       [0.51269531],
       [0.21142578],
       [0.0916748 ],
       [0.19750977],
       [0.57373047],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.60351562],
       [0.28857422],
       [0.        ],
       [0.70849609],
       [0.48413086],
       [0.41699219],
       [0.28710938]])

In [44]:
submission = Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1016